In [42]:
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
import pandas as pd


In [43]:
pytrends = TrendReq(hl='en-US', tz=360)


In [59]:
num_keywords = min(25, int(input("Enter the number of keywords (up to 25): ")))
keywords = [input(f"Enter keyword {i+1}: ") for i in range(num_keywords)]


Enter the number of keywords (up to 25): 4
Enter keyword 1: SALES SAAS
Enter keyword 2: BUSINESS SAAS
Enter keyword 3: B2B SAAS
Enter keyword 4: AI SAAS


In [60]:
pytrends.build_payload(keywords, timeframe='today 5-y', geo='', gprop='')
interest_over_time = pytrends.interest_over_time()


In [61]:
import plotly.graph_objects as go

def plot_interest_over_time(data, keys):
    fig = go.Figure()
    
    for key in keys:
        if key in data.columns:
            fig.add_trace(go.Scatter(x=data.index, y=data[key], mode='lines', name=key, hoverinfo='x+y+name'))
    
    fig.update_layout(
        title='Google Trends Over Time',
        xaxis_title='Date',
        yaxis_title='Interest',
        xaxis=dict(showline=True, showgrid=True, showticklabels=True, linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)')),
        yaxis=dict(showgrid=True, zeroline=False, showline=False, showticklabels=True),
        autosize=True,
        margin=dict(autoexpand=True, l=100, r=20, t=110),
        showlegend=True
    )

    fig.show()

plot_interest_over_time(interest_over_time, keywords)


In [82]:
# Ensure only numeric columns are considered
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')  # Dropping the 'isPartial' column if present
highest_interest = numeric_data.max()
highest_interest_keywords = highest_interest.idxmax() if not highest_interest.empty else "No data available"

time_intervals = ['1w', '1m', '3m', '6m', '12m']
percent_changes = {}

for interval in time_intervals:
    interval_data = interest_over_time.last(interval).iloc[0]
    # Calculate percentage change for each keyword
    percent_change = (interval_data - numeric_data.iloc[0]) / numeric_data.iloc[0] * 100
    percent_changes[interval] = percent_change

# Convert the dictionary to a DataFrame
percent_change_df = pd.DataFrame(percent_changes)
percent_change_df.head()


,1w,1m,3m,6m,12m
AI SAAS,760.0,680.0,720.0,460.0,100.0
B2B SAAS,500.0,400.0,423.076923,330.769231,330.769231
BUSINESS SAAS,194.117647,126.470588,91.176471,94.117647,102.941176
SALES SAAS,53.333333,60.0,77.777778,55.555556,62.222222
isPartial,NaN,NaN,NaN,NaN,NaN


In [87]:
import plotly.express as px

# Drop the 'isPartial' column from numeric_data
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')

# Drop the 'isPartial' column from percent_change_df
percent_change_df = percent_change_df.dropna()
# Create a heatmap using Plotly Express
fig = px.imshow(percent_change_df,
                x=percent_change_df.columns,
                y=percent_change_df.index,
                color_continuous_scale='',
                title='Percentage Increase in Interest Over Time Intervals - Heatmap')

# Update the layout
fig.update_layout(
    xaxis=dict(side='top'),
    xaxis_title='Keywords',
    yaxis_title='Time Intervals',
    autosize=True,
    margin=dict(autoexpand=True, l=100, r=20, t=110),
    showlegend=True
)

# Show the plot
fig.show()


In [66]:
percent_change_df.head()

NameError: name 'percent_change_df' is not defined